<a href="https://colab.research.google.com/github/alexhein189/code_snippets/blob/master/cnn_cifar_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.nn as nn
import torch
from torchvision import transforms, datasets
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time


In [ ]:
train_data = datasets.CIFAR10(root= "/content/sample_data", train=True, transform=transforms.ToTensor(),download=True)
test_data = datasets.CIFAR10(root= "/content/sample_data", train=False, transform=transforms.ToTensor(),download=True)


Extracting /content/sample_data/cifar-10-python.tar.gz to /content/sample_data
Files already downloaded and verified


In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=False)

In [ ]:
for img, label in train_loader:
  print(img.size(), label)
  break

torch.Size([32, 3, 32, 32]) tensor([7, 2, 5, 3, 6, 5, 2, 9, 3, 4, 7, 9, 7, 6, 1, 4, 1, 6, 5, 1, 0, 1, 6, 7,
        7, 7, 3, 8, 2, 8, 4, 2])


CNN class

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    #initial image size = 32*32*3
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1)
    #the size after conv1 is 30*30*16
    self.pool = nn.MaxPool2d(2)
    #the size after pooling is 15*15*16
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1)
    #the size after conv2 is 13*13*32
    #the size after pooling is 7*7*32

    self.fc1 = nn.Linear(6*6*32, 256)
    self.fc2 = nn.Linear(256, 64)
    self.fc3 = nn.Linear(64, 32)
    self.fc4 = nn.Linear(32, 10)

    self.relu = nn.ReLU()


  def forward(self,x):
    x = self.conv1(x)
    x = self.relu(x)
    x = self.pool(x)

    x = self.conv2(x)
    x = self.relu(x)
    x = self.pool(x)

    out = x.view(-1, 6*6*32)

    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.relu(out)
    out = self.fc3(out)
    out = self.relu(out)
    out = self.fc4(out)

    return out

    

In [ ]:
model = CNN()
loss_fn = nn.CrossEntropyLoss()
optimiser = optim.SGD(params=model.parameters(), lr=0.001)

In [ ]:
def train(model, loss_fn, optimiser, epochs=10):
  training_loss = []
  testing_loss = []

  training_accuracy = []
  testing_accuracy = []

  for epoch in range(epochs):
    start = time.time()
    train_loss, train_accuracy = 0.0, 0.0
    
    model.train()
    
    for img, label in train_loader:
      optimiser.zero_grad()
      output = model(img)

      loss = loss_fn(output, label)
      loss.backward()

      optimiser.step()

      train_loss += loss.data.item()
      training_loss.append(train_loss)

      prediction = torch.argmax(output, 1)
      train_accuracy += (prediction == label).sum().item()
      training_accuracy.append(train_accuracy)

    
    test_loss, test_accuracy = 0.0, 0.0
    model.eval()

    with torch.no_grad():
      for img, label in test_loader:
        output = model(img)
        loss = loss_fn(output, label)

        test_loss += loss.data.item()
        testing_loss.append(test_loss)

        prediction = torch.argmax(output, 1)
        test_accuracy += (prediction == label).sum().item()
        testing_accuracy.append(test_accuracy)
    
    end = time.time()
    print('Epoch: {}/{} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Accuracy: {:.2f} Test Loss: {:.2f} - {:.2f}secs'.format(
            epoch+1, epochs, train_loss, train_accuracy, test_accuracy, test_loss, end-start)) 
    
  
  plt.plot(range(training_accuracy), training_accuracy, label="train accuracy")
  plt.plot(range(testing_accuracy), testing_accuracy, label="test accuracy")
  plt.show()

  plt.plot(range(training_loss), training_loss, label="train loss")
  plt.plot(range(testing_loss), testing_loss, label="test loss")
  plt.show()
  

In [ ]:
train(model,loss_fn, optimiser)

Epoch: 1/10 Train Loss: 3557.05 Train Accuracy: 7589.00 Test Accuracy: 7552.00 Test Loss: 3546.12 - 47.65secs
Epoch: 2/10 Train Loss: 3529.62 Train Accuracy: 7585.00 Test Accuracy: 7729.00 Test Loss: 3508.78 - 47.77secs
Epoch: 3/10 Train Loss: 3477.52 Train Accuracy: 7825.00 Test Accuracy: 8225.00 Test Loss: 3439.73 - 48.45secs
Epoch: 4/10 Train Loss: 3393.20 Train Accuracy: 8722.00 Test Accuracy: 9442.00 Test Loss: 3344.65 - 47.77secs
Epoch: 5/10 Train Loss: 3303.70 Train Accuracy: 9715.00 Test Accuracy: 10215.00 Test Loss: 3268.93 - 47.83secs
Epoch: 6/10 Train Loss: 3243.85 Train Accuracy: 10630.00 Test Accuracy: 11150.00 Test Loss: 3225.39 - 48.00secs
Epoch: 7/10 Train Loss: 3208.67 Train Accuracy: 11395.00 Test Accuracy: 11242.00 Test Loss: 3190.87 - 47.56secs
Epoch: 8/10 Train Loss: 3181.07 Train Accuracy: 11635.00 Test Accuracy: 11794.00 Test Loss: 3182.15 - 47.42secs
Epoch: 9/10 Train Loss: 3154.42 Train Accuracy: 11947.00 Test Accuracy: 12111.00 Test Loss: 3139.82 - 47.79secs
E

TypeError: ignored